In [62]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



**This script performs a correlation between two implicit (click-though data) and explicit (human relevance judgement) feedback for an Enterprise Search (ES) service that uses Learning to Rank (LTR)**

In [63]:
# Much of the code in this notebook inspired by 
# https://towardsdatascience.com/compare-multiple-frequency-distributions-to-extract-valuable-information-from-a-dataset-10cba801f07b
# 

In [64]:
#!pip3 install -U datapane >/dev/null 2>&1

In [65]:
#!pip3 install -U jinja2

In [66]:
import seaborn as sns
import altair as alt 
import pandas as pd


exim0=pd.read_csv("/kaggle/input/explicitimplicitv5/explicit-implicit-v5.csv")

exim=pd.read_csv("/kaggle/input/entrp-srch/ENTRP-SRCH-v12.txt",sep='\s+', header=None, names=["HUMAN", "QID", "BM25", "RECENCY", "CONTACT", "ABOUT", "RAWHITS","URL-LEN","LINKRANK","CTR-6"])

In [67]:
exim['CTR-6'] = exim['CTR-6'].str[2:]

In [68]:
#exim["HUMAN"] = pd.to_numeric(exim["HUMAN"])
# convert just columns "a" and "b"
exim[["HUMAN", "CTR-6"]] = exim[["HUMAN", "CTR-6"]].apply(pd.to_numeric)


In [69]:
exim.head(5)

In [70]:
# eclude those documents that have no (NULL) CTR score 
exim = exim[exim["CTR-6"] != 0.00]

In [71]:
exim.describe

In [72]:
#exim.describe

In [73]:
#exim.info

In [74]:
import seaborn as sns

import matplotlib.pyplot as plt


In [75]:
#sns.stripplot(x = 'Pos', y = 'Height', data = wnba)
sns.set(rc={'figure.figsize':(15,10)})
sns.set(font_scale=2)

#sns.stripplot(x = 'HUMAN', y = 'CLICK-THRU-6', data = exim).set(title='Strip Plot of raw Click-through versus human relevance judgements',xlabel='human relevance judgements',ylabel='raw Click-through 6 months')

In [76]:
sns.set(rc={'figure.figsize':(15,10)})
sns.set(font_scale=2)

sns.stripplot(x = 'HUMAN', y = 'CTR-6', data = exim).set(title='Strip Plot of CTR versus Human relevance judgements',xlabel='Human relevance judgements',ylabel='CTR %')

In [77]:
#sns.boxplot(x = 'Pos', y = 'Height', data = wnba)
#sns.boxplot(x = 'HUMAN', y = 'CLICK-THRU-6', data = exim)

In [78]:
#sns.set(rc={'figure.figsize':(11.7,8.27)})
#sns.boxplot(x = 'Pos', y = 'Height', data = wnba, whis=4)
#sns.boxplot(x = 'HUMAN', y = 'CLICK-THRU-6', data = exim, whis=4)

In [79]:
sns.boxplot(x = 'HUMAN', y = 'CTR-6', data = exim, whis=4).set(title='Box Plot of CTR versus Human relevance judgements',xlabel='Human relevance judgements',ylabel='CTR %')

In [80]:
!pip3 install scipy

In [81]:
from scipy.stats import ttest_rel

In [82]:
#exim.rename(columns={'HUMAN':'','CTR-6:''})

In [83]:

# calculate the Pearson's correlation between two variables
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
from scipy.stats import spearmanr
# seed random number generator
seed(1)
# prepare data

In [84]:
#data1 = exim['HUMAN']
#data2 = exim['CLICK-THRU-6']#

#data1 = data1.dropna()
#data2 = data2.dropna()

# calculate Pearson's correlation
#corr, _ = pearsonr(data1, data2)
#print('Pearsons correlation human v raw click-through: %.3f' % corr)

In [85]:
data1 = exim['HUMAN']
data2 = exim['CTR-6']

#data1 = data1.dropna()
#data2 = data2.dropna()

# calculate Pearson's correlation
corr, _ = pearsonr(data1, data2)
print('Pearsons correlation human v CTR : %.3f' % corr)

In [86]:
data1 = exim['HUMAN']
data2 = exim['CTR-6']

#data1 = data1.dropna()
#data2 = data2.dropna()

from scipy.stats import spearmanr
corr, _ = spearmanr(data1, data2)
print('Spearmans correlation:  human v click-through-rate: %.3f' % corr)

In [87]:
# calculate spearman's correlation
coef, p = spearmanr(data1, data2)
print('Spearmans correlation coefficient: %.3f' % coef)
print ('p-value',p)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

In [88]:
#another way to report the results from the t-test by creating a violin plot:
#sns.violinplot(x='HUMAN', y='CLICK-THRU-6', bw=10, data=exim)

In [89]:
#another way to report the results from the t-test by creating a violin plot:
sns.violinplot(x='HUMAN', y='CTR-6', bw=2, data=exim)